In [1]:
# ==================================================================================================
# --- Imports
# ==================================================================================================
import tree_maker
import yaml
import pandas as pd
import time
import logging

# ==================================================================================================
# --- Load tree of jobs
# ==================================================================================================

# Start of the script
print("Analysis of output simulation files started")
start = time.time()

# Load Data
study_name = "tune_scan_new_RT_repro"
fix = "/scans/" + study_name
root = tree_maker.tree_from_json(fix[1:] + "/tree_maker.json")
# Add suffix to the root node path to handle scans that are not in the root directory
root.add_suffix(suffix=fix)


# ==================================================================================================
# --- # Browse simulations folder and extract relevant observables
# ==================================================================================================
l_problematic_sim = []
l_df_to_merge = []
for node in root.generation(1):
    with open(f"{node.get_abs_path()}/config.yaml", "r") as fid:
        config_parent = yaml.safe_load(fid)
    for node_child in node.children:
        with open(f"{node_child.get_abs_path()}/config.yaml", "r") as fid:
            config_child = yaml.safe_load(fid)

        try:
            # Read the particle path as relative
            try:
                particle = pd.read_parquet(
                    f"{node_child.get_abs_path()}/{config_child['config_simulation']['particle_file']}"
                )

            # If it doesn't work, try to read it as absolute
            except:
                particle = pd.read_parquet(f"{config_child['config_simulation']['particle_file']}")

            df_sim = pd.read_parquet(f"{node_child.get_abs_path()}/output_particles.parquet")

        except Exception as e:
            print(e)
            l_problematic_sim.append(node_child.get_abs_path())
            continue

        # Register paths and names of the nodes
        df_sim["path base collider"] = f"{node.get_abs_path()}"
        df_sim["name base collider"] = f"{node.name}"
        df_sim["path simulation"] = f"{node_child.get_abs_path()}"
        df_sim["name simulation"] = f"{node_child.name}"

        # Get node parameters as dictionnaries for parameter assignation
        dic_child_collider = node_child.parameters["config_collider"]
        dic_child_simulation = node_child.parameters["config_simulation"]
        try:
            dic_parent_collider = node.parameters["config_mad"]
        except:
            print("No parent collider could be loaded")
        dic_parent_particles = node.parameters["config_particles"]

        # Get which beam is being tracked
        df_sim["beam"] = dic_child_simulation["beam"]

        # Get scanned parameters (complete with the requested scanned parameters)
        df_sim["qx"] = dic_child_collider["config_knobs_and_tuning"]["qx"]["lhcb1"]
        df_sim["qy"] = dic_child_collider["config_knobs_and_tuning"]["qy"]["lhcb1"]
        df_sim["dqx"] = dic_child_collider["config_knobs_and_tuning"]["dqx"]["lhcb1"]
        df_sim["dqy"] = dic_child_collider["config_knobs_and_tuning"]["dqy"]["lhcb1"]
        df_sim["i_bunch_b1"] = dic_child_collider["config_beambeam"]["mask_with_filling_pattern"][
            "i_bunch_b1"
        ]
        df_sim["i_bunch_b2"] = dic_child_collider["config_beambeam"]["mask_with_filling_pattern"][
            "i_bunch_b2"
        ]
        df_sim["num_particles_per_bunch"] = dic_child_collider["config_beambeam"][
            "num_particles_per_bunch"
        ]
        df_sim["i_oct_b1"] = dic_child_collider["config_knobs_and_tuning"]["knob_settings"][
            "i_oct_b1"
        ]
        df_sim["i_oct_b2"] = dic_child_collider["config_knobs_and_tuning"]["knob_settings"][
            "i_oct_b2"
        ]
        df_sim["crossing_angle"] = abs(
            float(dic_child_collider["config_knobs_and_tuning"]["knob_settings"]["on_x1"])
        )

        # Merge with particle data
        df_sim_with_particle = pd.merge(df_sim, particle, on=["particle_id"])
        l_df_to_merge.append(df_sim_with_particle)

# ==================================================================================================
# --- # Merge all jobs outputs in one dataframe and save it
# ==================================================================================================

# Merge the dataframes from all simulations together
df_all_sim = pd.concat(l_df_to_merge)

# Extract the particles that were lost for DA computation
df_lost_particles = df_all_sim[df_all_sim["state"] != 1]  # Lost particles

# Check if the dataframe is empty
if df_lost_particles.empty:
    print("No unstable particles found, the output dataframe will be empty.")

# Group by working point (Update this with the knobs you want to group by !)
group_by_parameters = ["name base collider", "qx", "qy"]
# We always want to keep beam in the final result
group_by_parameters = ["beam"] + group_by_parameters
l_parameters_to_keep = [
    "normalized amplitude in xy-plane",
    "qx",
    "qy",
    "dqx",
    "dqy",
    "i_bunch_b1",
    "i_bunch_b2",
    "num_particles_per_bunch",
    "crossing_angle",
]

# Min is computed in the groupby function, but values should be identical
my_final = pd.DataFrame(
    [
        df_lost_particles.groupby(group_by_parameters)[parameter].min()
        for parameter in l_parameters_to_keep
    ]
).transpose()

Analysis of output simulation files started


In [2]:
my_final

,,,,normalized amplitude in xy-plane,qx,qy,dqx,dqy,i_bunch_b1,i_bunch_b2,num_particles_per_bunch,crossing_angle
beam,name base collider,qx,qy,,,,,,,,,
lhcb1,base_collider,62.314,60.319,5.84375,62.314,60.319,15.0,15.0,411.0,410.0,1.224536e+11,160.0


In [3]:
df_all_sim

,start_tracking_at_element,q0,mass0,s,zeta,chi,charge_ratio,weight,particle_id,at_element,...,dqx,dqy,i_bunch_b1,i_bunch_b2,num_particles_per_bunch,i_oct_b1,i_oct_b2,crossing_angle,normalized amplitude in xy-plane,angle in xy-plane [deg]
0,-1,1.0,9.382721e+08,0.000000,0.122295,1.0,1.0,0.0,0,0,...,15.0,15.0,411,410,1.224536e+11,300.0,300.0,160.0,4.50000,15.0
1,-1,1.0,9.382721e+08,0.000000,0.145099,1.0,1.0,0.0,1,0,...,15.0,15.0,411,410,1.224536e+11,300.0,300.0,160.0,4.53125,15.0
2,-1,1.0,9.382721e+08,0.000000,0.111039,1.0,1.0,0.0,2,0,...,15.0,15.0,411,410,1.224536e+11,300.0,300.0,160.0,4.56250,15.0
0,-1,1.0,9.382721e+08,0.000000,0.118488,1.0,1.0,0.0,3,0,...,15.0,15.0,411,410,1.224536e+11,300.0,300.0,160.0,4.59375,15.0
1,-1,1.0,9.382721e+08,0.000000,0.137614,1.0,1.0,0.0,4,0,...,15.0,15.0,411,410,1.224536e+11,300.0,300.0,160.0,4.62500,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,-1,1.0,9.382721e+08,8186.479686,-0.438139,1.0,1.0,0.0,479,8682,...,15.0,15.0,411,410,1.224536e+11,300.0,300.0,160.0,7.34375,75.0
0,-1,1.0,9.382721e+08,0.000000,0.225631,1.0,1.0,0.0,482,0,...,15.0,15.0,411,410,1.224536e+11,300.0,300.0,160.0,7.43750,75.0
1,-1,1.0,9.382721e+08,8295.765988,-0.328366,1.0,1.0,0.0,481,8770,...,15.0,15.0,411,410,1.224536e+11,300.0,300.0,160.0,7.40625,75.0
0,-1,1.0,9.382721e+08,15281.348060,0.244284,1.0,1.0,0.0,483,15448,...,15.0,15.0,411,410,1.224536e+11,300.0,300.0,160.0,7.46875,75.0
